In [3]:
import numpy as np
import pydae.build as db
import sympy as sym

In [56]:
class cig:
    
    def __init__(self, data):

        self.f_list = []
        self.x_list = []
        self.g_list = []
        self.y_ini_list = []
        self.y_run_list = []
        self_u_dict = {}
        self_h_dict = {}
        self.params_dict = {}
        
        self.data = data
        self.cigs = data['cigs']
        
    def build(self):
        
        cigs = self.cigs 
        for cig in cigs:
            self.vsc_lcl(cig)
            self.f_list += self.f_vsc_filter
            self.x_list += self.x_vsc_filter 
            self.g_list += self.g_vsc_filter 
            self.y_ini_list += self.y_vsc_filter 
            self.y_run_list += self.y_vsc_filter       

            self.transformations(cig)
            self.g_list += self.g_aux 
            self.y_ini_list += self.y_aux 
            self.y_run_list += self.y_aux    
            
            
    def vsg2dae(data,grid_dae):
        '''


        Parameters
        ----------
        data : TYPE
            DESCRIPTION.
        grid_dae : TYPE
            DESCRIPTION.

        Returns
        -------
        dict
            DESCRIPTION.

        '''

        sin = sym.sin
        cos = sym.cos
        sqrt = sym.sqrt 


        vsgs = data['vsgs']
        N_vsg = len(vsgs)

        # secondary control
        p_sec = {}
        q_sec = {}
        xi_f_sec,xi_v_sec = sym.symbols('xi_f_sec,xi_v_sec', real=True)
        K_f_sec,K_v_sec = sym.symbols('K_f_sec,K_v_sec', real=True)

        omega_coi_h = 0
        H_total = 0
        N_voltage = 0
        v_prom = 0
        for vsg in vsgs:
            name = vsg['name']

            omega_v_i  = sym.Symbol(f'omega_v_{name}', real=True)
            H = sym.Symbol(f'H_{name}', real=True)

            omega_coi_h += omega_v_i*H
            H_total += H 

            v_s_filt_i = sym.symbols(f'v_s_filt_{name}', real=True)
            N_voltage += 1
            v_prom += v_s_filt_i/N_vsg

        omega_coi = omega_coi_h/H_total
        dxi_f_sec = 1 - omega_coi
        dxi_v_sec = 1 - v_prom

        for vsg in vsgs:
            name = vsg['name']
            p_sec.update({f'{name}':K_f_sec*xi_f_sec/N_vsg})
            q_sec.update({f'{name}':K_v_sec*xi_v_sec/N_vsg})

        f_ctrl_5 = [dxi_f_sec,dxi_v_sec]
        x_ctrl_5 = [ xi_f_sec, xi_v_sec]
        g_ctrl_5 = []
        y_ctrl_5 = []
        x_0_ctrl_5 = []
        y_0_ctrl_5 = []
        params_ctrl_5 = {'K_f_sec':0.001,'K_v_sec':0.01}
        u_ctrl_5 = {}
        h_ctrl_5 = {}


        f_vsg = []
        x_vsg = []
        g_vsg = []
        y_vsg = []
        x_0_vsg = []
        y_0_vsg = []
        params_vsg = {}
        u_vsg = {}
        h_vsg = {}

        for vsg in vsgs:
            name = vsg['name']
            bus = vsg['bus']

            U_b = vsg['U_b']
            S_b = vsg['S_b_kVA']*1000
            I_b = S_b/(np.sqrt(3)*U_b)

            U_bdq = U_b*(np.sqrt(2))
            V_bdq = U_bdq/np.sqrt(3)
            I_bdq = I_b*np.sqrt(2)



    def transformations(self):
            ## Transformations #########################################################################
            ## feedbacks:
            feedbacks = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ'] + ['phi']
            for item in feedbacks:
                exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    



            v_md,v_mq = sym.symbols(f'v_md_{name},v_mq_{name}', real=True)
            v_sd,v_sq = sym.symbols(f'v_sd_{name},v_sq_{name}', real=True)
            i_sd,i_sq = sym.symbols(f'i_sd_{name},i_sq_{name}', real=True)
            i_td,i_tq = sym.symbols(f'i_td_{name},i_tq_{name}', real=True)
            phi,dum= sym.symbols(f'phi_{name},dum_{name}', real=True)

            eq_v_md = -v_md + v_mD*cos(phi) + v_mQ*sin(phi) # original park
            eq_v_mq = -v_mq - v_mD*sin(phi) + v_mQ*cos(phi) # original park

            eq_v_sd = -v_sd + v_sD*cos(phi) + v_sQ*sin(phi) # original park
            eq_v_sq = -v_sq - v_sD*sin(phi) + v_sQ*cos(phi) # original park

            eq_i_sd = -i_sd + i_sD*cos(phi) + i_sQ*sin(phi) # original park
            eq_i_sq = -i_sq - i_sD*sin(phi) + i_sQ*cos(phi) # original park

            # jmm: medimos i_t?
            eq_i_td = -i_td + i_tD*cos(phi) + i_tQ*sin(phi) # original park
            eq_i_tq = -i_tq - i_tD*sin(phi) + i_tQ*cos(phi) # original park

            g_aux = [eq_v_md,eq_v_mq,eq_v_sd,eq_v_sq,eq_i_td,eq_i_tq,eq_i_sd,eq_i_sq]    
            y_aux = [   v_md,   v_mq,   v_sd,   v_sq,   i_td,   i_tq,   i_sd,   i_sq]
            y_0_aux = [  0.0,  V_bdq,    0.0,  V_bdq,       0,      0]    


            #v_sd = v_md
            #v_sq = v_mq


           # S_b_kVA,U_b = sym.symbols(f'S_b_kVA_{name},U_b_{name}', real=True) # params



            ## per unit

            i_sd_pu = i_sd/I_bdq; # input in SI that is coverted to pu
            i_sq_pu = i_sq/I_bdq; # input in SI that is coverted to pu
            v_sd_pu = v_sd/V_bdq; # input in SI that is coverted to pu
            v_sq_pu = v_sq/V_bdq; # input in SI that is coverted to pu
            i_td_pu = i_td/I_bdq; # input in SI that is coverted to pu
            i_tq_pu = i_tq/I_bdq; # input in SI that is coverted to pu



            ## PLL #########################################################################

    def ctrl_4(self):
            # CTRL4 #########################################################################
            ## parameters:
            params_ctrl_4 = {}
            for item in ['T_vpoi','K_vpoi','T_f','K_f']:
                params_ctrl_4.update({f'{item}_{name}':vsg[item]})
                exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
            ## inputs:
            u_ctrl_4 = {}
            for item in ['v_s_ref','omega_ref','p_r','q_r']:
                u_ctrl_4.update({f'{item}_{name}':vsg[item]})
                exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
            ## dynamic states:
            x_list_ctrl_4 = ['omega_v_filt','v_s_filt']
            for item in x_list_ctrl_4:
                exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
            ## algebraic states
            y_list_ctrl_4 = ['p_m_ref','q_s_ref']
            for item in y_list_ctrl_4:
                exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    
            ## feedbacks:
            feedbacks = ['omega_v']
            for item in feedbacks:
                exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())       

            domega_v_filt = 1/T_f*(omega_v - omega_v_filt)
            dv_s_filt = 1/T_vpoi*((v_sd_pu**2+v_sq_pu**2)**0.5 - v_s_filt)

            eq_p_m_ref = -p_m_ref + p_r + K_f*(omega_ref - omega_v_filt) + p_sec[name]   # PFR and secondary input 
            eq_q_s_ref = -q_s_ref + q_r + K_vpoi*(v_s_ref - v_s_filt)    + q_sec[name]


            # from derivatives to the integrator
            f_ctrl_4 = [domega_v_filt,dv_s_filt];
            x_ctrl_4 = [ omega_v_filt, v_s_filt];
            g_ctrl_4 = [ eq_p_m_ref, eq_q_s_ref] #eq_i_sd_ref, eq_i_sq_ref, ]
            y_ctrl_4 = [    p_m_ref,    q_s_ref] #   i_sd_ref,    i_sq_ref,    omega_v,    DV_sat];




            # CTRL3 #########################################################################

    def ctrl_3(self):
            if vsg['ctrl3'] == 'uvsg_i':

                ## parameters:
                params_ctrl_3 = {}
                for item in ['K_p','T_p','K_q','T_q','R_v','X_v','S_b_kVA','U_b','K_phi','H','D']:
                    params_ctrl_3.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## inputs:
                u_ctrl_3 = {}
                for item in ['p_m_ref','q_s_ref']: # []
                    u_ctrl_3.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## dynamic states:
                x_list_ctrl_3 = ['phi','omega_v','xi_q','omega_rads']
                for item in x_list_ctrl_3:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## algebraic states
                y_list_ctrl_3 = ['DV_sat','p_s_pu','q_s_pu']
                for item in y_list_ctrl_3:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    
                ## feedbacks:   
                v_dc,dum = sym.symbols(f'v_dc_{name},dum_{name}', real=True)

                # equations:
                eq_omega_v = -omega_v + K_p*(epsilon_p + xi_p/T_p) + 1.0; 

                dphi =  Omega_b*(omega_v-1.0) - K_phi*phi;
                dxi_p = epsilon_p;
                dxi_q = epsilon_q; 
                domega_rads = 1.0/1.0*(Omega_b*omega_v - omega_rads);


            if vsg['ctrl3'] == 'sm2':

                ## parameters:
                params_ctrl_3 = {}
                for item in ['K_p','T_p','K_q','T_q','R_v','X_v','S_b_kVA','U_b','K_phi','H','D']:
                    params_ctrl_3.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## inputs:
                u_ctrl_3 = {}
                for item in ['p_m_ref','q_s_ref']: # []
                    u_ctrl_3.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## dynamic states:
                x_list_ctrl_3 = ['phi','omega_v','xi_q','omega_rads']
                for item in x_list_ctrl_3:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## algebraic states
                y_list_ctrl_3 = ['DV_sat','p_s_pu','q_s_pu']
                for item in y_list_ctrl_3:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    
                ## feedbacks:   
                v_dc,dum = sym.symbols(f'v_dc_{name},dum_{name}', real=True)


                # equations:

                fault_flag = 0;
                e_0 = 1.0;    
                eq_p_s_pu = -p_s_pu + i_sd_pu*v_sd_pu + i_sq_pu*v_sq_pu; # pu
                eq_q_s_pu = -q_s_pu + i_sd_pu*v_sq_pu - i_sq_pu*v_sd_pu; # pu 

                # from the integrator to the states
                epsilon_p = (p_m_ref - p_s_pu)*(1.0-fault_flag);
                epsilon_q =  q_s_ref - q_s_pu;


                dphi =  Omega_b*(omega_v-omega_coi) - K_phi*phi;
                domega_v = 1/(2*H)*(p_m_ref - p_s_pu - D*(omega_v - 1.0))
                dxi_q = epsilon_q; 
                domega_rads = 1.0/1.0*(Omega_b*omega_v - omega_rads);

                DV =   K_q*(epsilon_q + xi_q/T_q); 

                eq_DV_sat = DV_sat - DV;
                #if DV_sat > 0.1 
                #    DV_sat = 0.1;
                #    dxi_q = 0.0;
                #end
                #if DV_sat < -0.1 
                #    DV_sat = -0.1;
                #    dxi_q = 0.0;
                #end   

                e = e_0 + DV_sat;   


                if (not 'ctrl1' in vsg) and (not 'ctrl2' in vsg):                      # CTRL3 over CTRL0
                    v_t_d_pu = 0.0
                    v_t_q_pu = e

                    v_t_d = (v_t_d_pu - R_v*i_sd_pu + X_v*i_sq_pu)*V_bdq
                    v_t_q = (v_t_q_pu - R_v*i_sq_pu - X_v*i_sd_pu)*V_bdq

                    eta_d_ref = v_t_d/v_dc*2
                    eta_q_ref = v_t_q/v_dc*2

                    # from derivatives to the integrator
                    f_ctrl_3 = [dphi,domega_v,dxi_q,domega_rads];
                    x_ctrl_3 = [ phi, omega_v, xi_q, omega_rads];
                    g_ctrl_3 = [ eq_DV_sat, eq_p_s_pu, eq_q_s_pu] #eq_i_sd_ref, eq_i_sq_ref, ]
                    y_ctrl_3 = [    DV_sat,    p_s_pu,    q_s_pu] #   i_sd_ref,    i_sq_ref,    omega_v,    DV_sat];
                    x_0_ctrl_3 = [ 0.0, 0.0, 0.0, 2*np.pi*50]
                    y_0_ctrl_3 = [ ] #     0.0,       V_bdq,          1]  


                if ('ctrl1' in vsg) and (not 'ctrl2' in vsg):                          # CTRL3 over CTRL1
                    i_sd_pu = -(X_v*e + R_v*v_sd_pu + X_v*v_sq_pu)/(R_v**2 + X_v**2)
                    i_sq_pu = -(R_v*e + R_v*v_sq_pu - X_v*v_sd_pu)/(R_v**2 + X_v**2);
                    eq_i_sd_ref = -i_sd_ref + i_sd_pu*I_bdq
                    eq_i_sq_ref = -i_sq_ref + i_sq_pu*I_bdq

                    # from derivatives to the integrator
                    f_ctrl_3 = [dphi,domega_v,dxi_q,domega_rads];
                    x_ctrl_3 = [ phi, omega_v, xi_q, omega_rads];
                    g_ctrl_3 = [ eq_DV_sat, eq_p_s_pu, eq_q_s_pu, eq_i_sd_ref, eq_i_sq_ref]
                    y_ctrl_3 = [    DV_sat,    p_s_pu,    q_s_pu,    i_sd_ref,    i_sq_ref];
                    x_0_ctrl_3 = [ 0.0, 0.0, 0.0, 2*np.pi*50]
                    y_0_ctrl_3 = [ ] #     0.0,       V_bdq,          1]     

                if ('ctrl1' in vsg) and ('ctrl2' in vsg):                          # CTRL3 over CTRL2
                    v_sd_ref_pu =    - R_v*i_sd_pu + X_v*i_sq_pu; 
                    v_sq_ref_pu = -e - R_v*i_sq_pu - X_v*i_sd_pu;
                    eq_v_sd_ref = -v_sd_ref + v_sd_ref_pu*V_bdq
                    eq_v_sq_ref = -v_sq_ref + v_sq_ref_pu*V_bdq

                    # from derivatives to the integrator
                    f_ctrl_3 = [dphi,domega_v,dxi_q,domega_rads];
                    x_ctrl_3 = [ phi, omega_v, xi_q, omega_rads];
                    g_ctrl_3 = [ eq_DV_sat, eq_p_s_pu, eq_q_s_pu, eq_v_sd_ref, eq_v_sq_ref]
                    y_ctrl_3 = [    DV_sat,    p_s_pu,    q_s_pu,    v_sd_ref,    v_sq_ref];
                    x_0_ctrl_3 = [ 0.0, 0.0, 0.0, 2*np.pi*50]
                    y_0_ctrl_3 = [ ] #     0.0,       V_bdq,          1]              


            if vsg['ctrl3'] == 'droop_pq':

                ## parameters:
                params_ctrl_3 = {}
                for item in ['Omega_b','K_p','T_p','K_q','T_q','R_v','X_v','S_b_kVA','U_b','K_phi','H','D','K_omega','K_v','T_omega']:
                    params_ctrl_3.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## inputs:
                u_ctrl_3 = {}
                for item in ['p_m_ref','q_s_ref','e_ref']: # []
                    u_ctrl_3.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## dynamic states:
                x_list_ctrl_3 = ['phi','omega_v','xi_q','omega_rads']
                for item in x_list_ctrl_3:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## algebraic states
                y_list_ctrl_3 = ['p_s_pu','q_s_pu']
                for item in y_list_ctrl_3:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    
                ## feedbacks:   
                v_dc,dum = sym.symbols(f'v_dc_{name},dum_{name}', real=True)


                # equations:

                fault_flag = 0;
                e_0 = 1.0;    
                eq_p_s_pu = -p_s_pu + i_sd_pu*v_sd_pu + i_sq_pu*v_sq_pu; # pu
                eq_q_s_pu = -q_s_pu + i_sd_pu*v_sq_pu - i_sq_pu*v_sd_pu; # pu 

                # from the integrator to the states
                epsilon_p = (p_m_ref - p_s_pu)*(1.0-fault_flag);
                epsilon_q =  q_s_ref - q_s_pu;

                omega_v_ref = omega_ref + K_omega*epsilon_p 
                e =   e_ref + K_v*epsilon_q

                dphi =  Omega_b*(omega_v-omega_coi) - K_phi*phi;
                domega_v = 1/T_omega*(omega_v_ref - omega_v)
                domega_rads = 1.0/1.0*(Omega_b*omega_v - omega_rads);


                if (not 'ctrl1' in vsg) and (not 'ctrl2' in vsg):                      # CTRL3 over CTRL0
                    v_t_d_pu = 0.0
                    v_t_q_pu = e

                    v_t_d = (v_t_d_pu - R_v*i_sd_pu + X_v*i_sq_pu)*V_bdq
                    v_t_q = (v_t_q_pu - R_v*i_sq_pu - X_v*i_sd_pu)*V_bdq

                    eta_d_ref = v_t_d/v_dc*2
                    eta_q_ref = v_t_q/v_dc*2

                    # from derivatives to the integrator
                    f_ctrl_3 = [dphi,domega_v,domega_rads];
                    x_ctrl_3 = [ phi, omega_v, omega_rads];
                    g_ctrl_3 = [ eq_p_s_pu, eq_q_s_pu] #eq_i_sd_ref, eq_i_sq_ref, ]
                    y_ctrl_3 = [    p_s_pu,    q_s_pu] #   i_sd_ref,    i_sq_ref,    omega_v,    DV_sat];
                    x_0_ctrl_3 = [ 0.0, 0.0, 0.0, 2*np.pi*50]
                    y_0_ctrl_3 = [ ] #     0.0,       V_bdq,          1]  


                if ('ctrl1' in vsg) and (not 'ctrl2' in vsg):                          # CTRL3 over CTRL1
                    i_sd_pu = -(X_v*e + R_v*v_sd_pu + X_v*v_sq_pu)/(R_v**2 + X_v**2)
                    i_sq_pu = -(R_v*e + R_v*v_sq_pu - X_v*v_sd_pu)/(R_v**2 + X_v**2);
                    eq_i_sd_ref = -i_sd_ref + i_sd_pu*I_bdq
                    eq_i_sq_ref = -i_sq_ref + i_sq_pu*I_bdq

                    # from derivatives to the integrator
                    f_ctrl_3 = [dphi,domega_rads];
                    x_ctrl_3 = [ phi, omega_rads];
                    g_ctrl_3 = [ eq_DV_sat, eq_p_s_pu, eq_q_s_pu, eq_i_sd_ref, eq_i_sq_ref]
                    y_ctrl_3 = [    DV_sat,    p_s_pu,    q_s_pu,    i_sd_ref,    i_sq_ref];
                    x_0_ctrl_3 = [ 0.0, 0.0, 0.0, 2*np.pi*50]
                    y_0_ctrl_3 = [ ] #     0.0,       V_bdq,          1]     

                if ('ctrl1' in vsg) and ('ctrl2' in vsg):                          # CTRL3 over CTRL2
                    v_sd_ref_pu =    - R_v*i_sd_pu + X_v*i_sq_pu; 
                    v_sq_ref_pu = -e - R_v*i_sq_pu - X_v*i_sd_pu;
                    eq_v_sd_ref = -v_sd_ref + v_sd_ref_pu*V_bdq
                    eq_v_sq_ref = -v_sq_ref + v_sq_ref_pu*V_bdq

                    # from derivatives to the integrator
                    f_ctrl_3 = [dphi,domega_v,dxi_q,domega_rads];
                    x_ctrl_3 = [ phi, omega_v, xi_q, omega_rads];
                    g_ctrl_3 = [ eq_DV_sat, eq_p_s_pu, eq_q_s_pu, eq_v_sd_ref, eq_v_sq_ref]
                    y_ctrl_3 = [    DV_sat,    p_s_pu,    q_s_pu,    v_sd_ref,    v_sq_ref];
                    x_0_ctrl_3 = [ 0.0, 0.0, 0.0, 2*np.pi*50]
                    y_0_ctrl_3 = [ ] #     0.0,       V_bdq,          1]        

            if vsg['ctrl3'] == 'genape':
                dphi =  Omega_b*(omega_v-omega_coi) - K_phi*phi;
                domega_v = RoCoF/F_b;





    def ctrl_0(self):
            # CTRL0 #########################################################################

            ## inputs:
            params_ctrl_0 = {}
            u_ctrl_0 = {}
            for item in []: # ['eta_d_ref','eta_q_ref','phi']: #+['eta_D','eta_Q']:
                u_ctrl_0.update({f'{item}_{name}':vsg[item]})
                exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
            ## algebraic states
            y_list_ctrl_0 = ['eta_d','eta_q'] + ['eta_D','eta_Q'] 
            for item in y_list_ctrl_0:
                exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())        

            eq_eta_d = eta_d - eta_d_ref
            eq_eta_q = eta_q - eta_q_ref

            eq_eta_D = -eta_D + eta_d*cos(phi) - eta_q*sin(phi) # comment for test 1 
            eq_eta_Q = -eta_Q + eta_d*sin(phi) + eta_q*cos(phi) # comment for test 1

            # from derivatives to the integrator
            f_ctrl_0 = [];
            x_ctrl_0 = [];
            g_ctrl_0 = [eq_eta_d,eq_eta_q,eq_eta_D,eq_eta_Q]
            y_ctrl_0 = [   eta_d,   eta_q,   eta_D,   eta_Q];
            x_0_ctrl_0 = [ ]
            y_0_ctrl_0 = [0.0,  0.8, 0.0,  0.8]    



            ## VSC and Filters #########################################################################

            if vsg['filter'] == 'L':
                ## parameters:
                params_vsc_filter = {}
                for item in ['L_t','R_t','omega']:
                    params_vsc_filter.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## inputs:
                u_vsc_filter= {}
                for item in ['v_dc']: #+['eta_D','eta_Q']:
                    u_vsc_filter.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## dynamic states:
                x_list_vsc_filter = []
                for item in x_list_vsc_filter:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## algebraic states
                y_list_vsc_filter = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ']
                for item in y_list_vsc_filter:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")    
                ## feedbacks:
                v_poiD,v_poiQ = sym.symbols(f'v_{bus}_D,v_{bus}_Q', real=True)
                i_poiD,i_poiQ = sym.symbols(f'i_{bus}_D,i_{bus}_Q', real=True)

                #eta_D = eta_D_ref # - Gv_in*(i_tD - i_sD)
                #eta_Q = eta_Q_ref #- Gv_in*(i_tQ - i_sQ)

                # LCL filter
                di_tD = 1/L_t*(eta_D/2*v_dc - R_t*i_tD + omega*L_t*i_tQ - v_mD)  
                di_tQ = 1/L_t*(eta_Q/2*v_dc - R_t*i_tQ - omega*L_t*i_tD - v_mQ) 
                dv_mD = 1/C_m*(i_tD + C_m*omega*v_mQ - G_d*v_mD - i_sD) 
                dv_mQ = 1/C_m*(i_tQ - C_m*omega*v_mD - G_d*v_mQ - i_sQ) 
                di_sD = 1/L_s*(v_mD - R_s*i_sD + omega*L_s*i_sQ - v_sD)  
                di_sQ = 1/L_s*(v_mQ - R_s*i_sQ - omega*L_s*i_sD - v_sQ) 

                # Grid interaction
                eq_i_poiD =  i_sD - i_poiD
                eq_i_poiQ =  i_sQ - i_poiQ   

                eq_v_sD =  v_sD - v_poiD
                eq_v_sQ =  v_sQ - v_poiQ    

                grid_dae['params'].pop(f'i_{bus}_D')
                grid_dae['params'].pop(f'i_{bus}_Q')

                # DAE
                f_vsc_filter = []
                x_vsc_filter = []
                g_vsc_filter = [di_tD, di_tQ, eq_i_poiD, eq_i_poiQ, eq_v_sD, eq_v_sQ]
                y_vsc_filter = [ i_tD,  i_tQ,    i_poiD,    i_poiQ,    v_sD,    v_sQ]    
                x_0_vsc_filter = [ ]
                y_0_vsc_filter = [ 0.0,  0.0,       0.0,         0,       0,   V_bdq]

            if vsg['filter'] == 'LCL':
                ## parameters:
                params_vsc_filter = {}
                for item in ['L_t','R_t','C_m','L_s','R_s','omega','G_d']:
                    params_vsc_filter.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## inputs:
                u_vsc_filter = {}
                for item in ['v_dc']: #+['eta_D','eta_Q']:
                    u_vsc_filter.update({f'{item}_{name}':vsg[item]})
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
                ## dynamic states:
                x_list_vsc_filter = []
                for item in x_list_vsc_filter:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
                ## algebraic states
                y_list_vsc_filter = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ']
                for item in y_list_vsc_filter:
                    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    
                ## feedbacks:
                v_poiD,v_poiQ = sym.symbols(f'v_{bus}_D,v_{bus}_Q', real=True)
                i_poiD,i_poiQ = sym.symbols(f'i_{bus}_D,i_{bus}_Q', real=True)

                #eta_D = eta_D_ref # - Gv_in*(i_tD - i_sD)
                #eta_Q = eta_Q_ref #- Gv_in*(i_tQ - i_sQ)

                # LCL filter
                di_tD = 1/L_t*(eta_D/2*v_dc - R_t*i_tD + omega*L_t*i_tQ - v_mD)  
                di_tQ = 1/L_t*(eta_Q/2*v_dc - R_t*i_tQ - omega*L_t*i_tD - v_mQ) 
                dv_mD = 1/C_m*(i_tD + C_m*omega*v_mQ - G_d*v_mD - i_sD) 
                dv_mQ = 1/C_m*(i_tQ - C_m*omega*v_mD - G_d*v_mQ - i_sQ) 
                di_sD = 1/L_s*(v_mD - R_s*i_sD + omega*L_s*i_sQ - v_sD)  
                di_sQ = 1/L_s*(v_mQ - R_s*i_sQ - omega*L_s*i_sD - v_sQ) 

                # Grid interaction
                eq_i_poiD =  i_sD - i_poiD
                eq_i_poiQ =  i_sQ - i_poiQ   

                eq_v_sD =  v_sD - v_poiD
                eq_v_sQ =  v_sQ - v_poiQ    

                grid_dae['params'].pop(f'i_{bus}_D')
                grid_dae['params'].pop(f'i_{bus}_Q')

                # DAE
                f_vsc_filter = []
                x_vsc_filter = []
                g_vsc_filter = [di_tD, di_tQ, dv_mD, dv_mQ, di_sD, di_sQ, eq_i_poiD, eq_i_poiQ, eq_v_sD, eq_v_sQ]
                y_vsc_filter = [ i_tD,  i_tQ,  v_mD,  v_mQ,  i_sD,  i_sQ,    i_poiD,    i_poiQ,    v_sD,    v_sQ]    
                x_0_vsc_filter = [ ]
                y_0_vsc_filter = [ 0.0,  0.0,   0.0, V_bdq,     0,     0,         0,         0,       0,   V_bdq]


            ## Model integration
            f_vsg += f_vsc_filter + f_ctrl_0 + f_ctrl_3 + f_ctrl_4
            x_vsg += x_vsc_filter + x_ctrl_0 + x_ctrl_3 + x_ctrl_4
            g_vsg += g_vsc_filter + g_ctrl_0 + g_aux + g_ctrl_3 + g_ctrl_4  
            y_vsg += y_vsc_filter + y_ctrl_0 + y_aux + y_ctrl_3 + y_ctrl_4
            params_vsg.update(params_vsc_filter)
            params_vsg.update(params_ctrl_0)
            params_vsg.update(params_ctrl_3)
            params_vsg.update(params_ctrl_4)
            u_vsg.update(u_vsc_filter)
            u_vsg.update(u_ctrl_0)
            u_vsg.update(u_ctrl_3)
            u_vsg.update(u_ctrl_4)    

            h_vsg.update({f'i_sD_{name}':i_sD,f'i_sQ_{name}':i_sQ})

           # x_0_vsg += x_0_vsc_lc 
           # y_0_vsg += y_0_vsc_lc 

        f_vsg += f_ctrl_5
        x_vsg += x_ctrl_5
        params_vsg.update(params_ctrl_5)
        u_vsg.update(u_ctrl_5)

        return {'f_list':f_vsg,'g_list':g_vsg,
                'x_list':x_vsg,'y_list':y_vsg,
                'u_run_dict':u_vsg,'params_dict':params_vsg,'h_dict':h_vsg,
                'omega_coi':omega_coi}  


In [57]:
cig_obj = cig(data)
cig_obj.build()
cig_obj.x_list
cig_obj.y_ini_list

[i_1_D,
 i_1_Q,
 v_sD_g01,
 v_sQ_g01,
 v_md_g01,
 v_mq_g01,
 v_sd_g01,
 v_sq_g01,
 i_td_g01,
 i_tq_g01,
 i_sd_g01,
 i_sq_g01,
 eta_D_g01,
 eta_Q_g01]

In [44]:
Omega_b = 2*np.pi*50
L_t = 1.25e-3; #Converter side Filter Inductance H
R_t = L_t*Omega_b/10; #Converter side Filter resistance Ohms. Quality Factor of 10
L_s = 1.25e-3; #Grid Side Filter Inductance H
R_s = L_s*Omega_b/10; #Grid side Filter resistance Ohms. Quality Factor of 10
R_d = 20; #Dmaping resistor
C_m = 4e-6; #Filter capacitance
tau_ctrl_1 = 5e-3; #Time constant of CTRL 1
Ts_Power=50e-6; #Sample time of power plant
Ts_Control=50e-6; #Sample time of controller
K_pi = (L_t+L_s)/tau_ctrl_1; #Proportional gain of CTRL 1
K_ii = (R_t+R_s)/tau_ctrl_1; #Integral gain of CTRL 1
H = 5.0; # desired virtual inertia 
K_p = 0.01; # active power proportinal gain
#H = T_p/K_p/2

T_p = K_p*2*H;  # active power integral time constant

K_q = 0.1; # reactive power proportinal gain
T_q = 0.1; # reactive power integral time constant

X_v = 0.05; # virtual reactance
R_v = 0.01;  # virtual resistance

K_pv = 0.0004
K_iv = 11.313708

g01 = {"name":"g01","bus": "1","U_b":400.0,"S_b_kVA":30,"v_dc":750,'type':'sm2_i',
           'R_t':R_t,'L_t':L_t,'R_s':R_s,'L_s':L_s,'C_m':C_m,'K_phi':0.00,'omega':Omega_b, 'G_d':1.0, #VSC
           'T_eta':0.01,   # CTRL0
           'K_pi':K_pi,'K_ii':K_ii,'Gv_in':0.1,   # CTRL1
           'K_pv':K_pv,'K_iv':K_iv,   # CTRL2
           'K_p':K_p,'T_p':T_p,'K_q':K_q,'T_q':T_q,'R_v':R_v,'X_v':X_v,'p_m_ref':0.5,'q_s_ref':0.2,   # CTRL3
           'H':5.0,'D':5.0,  # CTRL3
           'K_f':20.0, 'K_vr':20.0}   # CTRL4

data = {
"system":{"f_hz":50,"model_type":"ae"},
"buses": [{"bus":"1"},{"bus":"2"}],
"lines":     [  {"bus_j":"1", "bus_k":"2", "R":0.01, "X":0.1, "B":1e-3}
             ],
"loads" : [
    #    {"bus": "R06", "kVA": 55.0, "pf": 0.95, "T_i":0.01,"I_max":1200}
        ],
"grid_formers" : [
                  {"bus": "1","V_phph":400.0, "deg":0.0},
                  {"bus": "2","V_phph":400.0, "deg":0.0},
                 ],
"cigs" : [g01]}


In [16]:
cos = sym.cos
sin = sym.sin

vsg = g01

name = vsg['name']
bus = vsg['bus']

U_b = vsg['U_b']
S_b = vsg['S_b_kVA']*1000
I_b = S_b/(np.sqrt(3)*U_b)

U_bdq = U_b*(np.sqrt(2))
V_bdq = U_bdq/np.sqrt(3)
I_bdq = I_b*np.sqrt(2)

eta_D,eta_Q =sym.symbols(f'eta_D_{name},eta_Q_{name}',real=True)

params_vsg = {}
u_vsg = {}

## Transformations #########################################################################
## feedbacks:
feedbacks = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ'] + ['phi']
for item in feedbacks:
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    

v_md,v_mq = sym.symbols(f'v_md_{name},v_mq_{name}', real=True)
v_sd,v_sq = sym.symbols(f'v_sd_{name},v_sq_{name}', real=True)
i_sd,i_sq = sym.symbols(f'i_sd_{name},i_sq_{name}', real=True)
i_td,i_tq = sym.symbols(f'i_td_{name},i_tq_{name}', real=True)
eta_d,eta_q = sym.symbols(f'eta_d_{name},eta_q_{name}', real=True)
phi,dum= sym.symbols(f'phi_{name},dum_{name}', real=True)

eq_v_md = -v_md + v_mD*cos(phi) + v_mQ*sin(phi) # original park
eq_v_mq = -v_mq - v_mD*sin(phi) + v_mQ*cos(phi) # original park

eq_v_sd = -v_sd + v_sD*cos(phi) + v_sQ*sin(phi) # original park
eq_v_sq = -v_sq - v_sD*sin(phi) + v_sQ*cos(phi) # original park

eq_i_sd = -i_sd + i_sD*cos(phi) + i_sQ*sin(phi) # original park
eq_i_sq = -i_sq - i_sD*sin(phi) + i_sQ*cos(phi) # original park

# jmm: medimos i_t?
eq_i_td = -i_td + i_tD*cos(phi) + i_tQ*sin(phi) # original park
eq_i_tq = -i_tq - i_tD*sin(phi) + i_tQ*cos(phi) # original park

eq_eta_D = -eta_D + eta_d*cos(phi) - eta_q*sin(phi) # original park
eq_eta_Q = -eta_Q + eta_d*sin(phi) + eta_q*cos(phi) # original park

g_aux = [eq_v_md,eq_v_mq,eq_v_sd,eq_v_sq,eq_i_td,eq_i_tq,eq_i_sd,eq_i_sq,eq_eta_D,eq_eta_Q]    
y_aux = [   v_md,   v_mq,   v_sd,   v_sq,   i_td,   i_tq,   i_sd,   i_sq,   eta_D,   eta_Q]
y_0_aux = [  0.0,  V_bdq,    0.0,  V_bdq,       0,      0]    
        
        
## parameters:
params_vsc_filter = {}
for item in ['L_t','R_t','C_m','L_s','R_s','omega','G_d']:
    params_vsc_filter.update({f'{item}_{name}':vsg[item]})
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
## inputs:
u_vsc_filter = {}
for item in ['v_dc']:#,'eta_D','eta_Q']:
    u_vsc_filter.update({f'{item}_{name}':vsg[item]})
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
## dynamic states:
x_list_vsc_filter = []
for item in x_list_vsc_filter:
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)")
## algebraic states
y_list_vsc_filter = ['i_tD','i_tQ'] + ['v_mD','v_mQ'] + ['i_sD','i_sQ'] + ['v_sD','v_sQ']
for item in y_list_vsc_filter:
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())    
## feedbacks:
v_poiD,v_poiQ = sym.symbols(f'v_{bus}_D,v_{bus}_Q', real=True)
i_poiD,i_poiQ = sym.symbols(f'i_{bus}_D,i_{bus}_Q', real=True)

#eta_D = eta_D_ref # - Gv_in*(i_tD - i_sD)
#eta_Q = eta_Q_ref #- Gv_in*(i_tQ - i_sQ)

# LCL filter
di_tD = 1/L_t*(eta_D/2*v_dc - R_t*i_tD + omega*L_t*i_tQ - v_mD)  
di_tQ = 1/L_t*(eta_Q/2*v_dc - R_t*i_tQ - omega*L_t*i_tD - v_mQ) 
dv_mD = 1/C_m*(i_tD + C_m*omega*v_mQ - G_d*v_mD - i_sD) 
dv_mQ = 1/C_m*(i_tQ - C_m*omega*v_mD - G_d*v_mQ - i_sQ) 
di_sD = 1/L_s*(v_mD - R_s*i_sD + omega*L_s*i_sQ - v_sD)  
di_sQ = 1/L_s*(v_mQ - R_s*i_sQ - omega*L_s*i_sD - v_sQ) 

# Grid interaction
eq_i_poiD =  i_sD - i_poiD
eq_i_poiQ =  i_sQ - i_poiQ   

eq_v_sD =  v_sD - v_poiD
eq_v_sQ =  v_sQ - v_poiQ    

#grid_dae['params'].pop(f'i_{bus}_D')
#grid_dae['params'].pop(f'i_{bus}_Q')

# DAE
f_vsc_filter = [di_tD, di_tQ, dv_mD, dv_mQ, di_sD, di_sQ]
x_vsc_filter = [ i_tD,  i_tQ,  v_mD,  v_mQ,  i_sD,  i_sQ]
g_vsc_filter = [ eq_i_poiD, eq_i_poiQ, eq_v_sD, eq_v_sQ]
y_vsc_filter = [    i_poiD,    i_poiQ,    v_sD,    v_sQ ]    
x_0_vsc_filter = [ ]
y_0_vsc_filter = [ 0.0,  0.0,   0.0, V_bdq,     0,     0,         0,         0,       0,   V_bdq]


## Model integration
f_vsg = f_vsc_filter 
x_vsg = x_vsc_filter
g_vsg = g_vsc_filter + g_aux
y_vsg = y_vsc_filter + y_aux
params_vsg.update(params_vsc_filter)
#params_vsg.update(params_ctrl_1)
u_vsg.update(u_vsc_filter)
#u_vsg.update(u_ctrl_0)
#u_vsg.update(u_ctrl_1)   
u_vsg.update({'v_1_D':0.0,'v_1_Q':326,'phi_g01':0.0,'eta_d_g01':0.0,'eta_q_g01':0.8})
#u_vsg.update({'v_1_D':0.0,'v_1_Q':326,'eta_d_g01':0.0,'eta_q_g01':0.8})

In [7]:
i_sd_g01,i_sq_g01 = sym.symbols(f'i_sd_g01,i_sq_g01', real=True)
sys_dict = {'name':'vsc_lcl',
           'params_dict':params_vsg,
           'f_list':[f_vsg],
           'g_list':[g_vsg],
           'x_list':[x_vsg],
           'y_ini_list':y_vsg,
           'y_run_list':y_vsg,
           'u_run_dict':u_vsg,
           'u_ini_dict':u_vsg,
           'h_dict':{'v_t_D':eta_D/2*v_dc,'v_t_Q':eta_Q/2*v_dc,'damp_D':G_d*v_mD,'damp_Q':G_d*v_mQ,
                     'i_sd_g01':i_sd,'i_sq_g01':i_sq,
                     'v_md_g01':v_md,'v_mq_g01':v_mq,
                     'i_td_g01':i_td,'i_tq_g01':i_tq,
                     'v_sd_g01':v_sd,'v_sq_g01':v_sq,}
           }

db.system(sys_dict)
db.sys2num(sys_dict)

In [11]:
u_vsg

{'v_dc_g01': 750,
 'v_1_D': 0.0,
 'v_1_Q': 326,
 'phi_g01': 0.0,
 'eta_d_g01': 0.0,
 'eta_q_g01': 0.8}

In [6]:
f_vsg

[(L_t_g01*i_tQ_g01*omega_g01 - R_t_g01*i_tD_g01 + eta_D_g01*v_dc_g01/2 - v_mD_g01)/L_t_g01,
 (-L_t_g01*i_tD_g01*omega_g01 - R_t_g01*i_tQ_g01 + eta_Q_g01*v_dc_g01/2 - v_mQ_g01)/L_t_g01,
 (C_m_g01*omega_g01*v_mQ_g01 - G_d_g01*v_mD_g01 - i_sD_g01 + i_tD_g01)/C_m_g01,
 (-C_m_g01*omega_g01*v_mD_g01 - G_d_g01*v_mQ_g01 - i_sQ_g01 + i_tQ_g01)/C_m_g01,
 (L_s_g01*i_sQ_g01*omega_g01 - R_s_g01*i_sD_g01 + v_mD_g01 - v_sD_g01)/L_s_g01,
 (-L_s_g01*i_sD_g01*omega_g01 - R_s_g01*i_sQ_g01 + v_mQ_g01 - v_sQ_g01)/L_s_g01]

In [7]:
sys_dict['g'].jacobian(sys_dict['u_run'])

Matrix([
[0,  0,  0,                                                0,            0,             0],
[0,  0,  0,                                                0,            0,             0],
[0, -1,  0,                                                0,            0,             0],
[0,  0, -1,                                                0,            0,             0],
[0,  0,  0,   -v_mD_g01*sin(phi_g01) + v_mQ_g01*cos(phi_g01),            0,             0],
[0,  0,  0,   -v_mD_g01*cos(phi_g01) - v_mQ_g01*sin(phi_g01),            0,             0],
[0,  0,  0,   -v_sD_g01*sin(phi_g01) + v_sQ_g01*cos(phi_g01),            0,             0],
[0,  0,  0,   -v_sD_g01*cos(phi_g01) - v_sQ_g01*sin(phi_g01),            0,             0],
[0,  0,  0,   -i_tD_g01*sin(phi_g01) + i_tQ_g01*cos(phi_g01),            0,             0],
[0,  0,  0,   -i_tD_g01*cos(phi_g01) - i_tQ_g01*sin(phi_g01),            0,             0],
[0,  0,  0,   -i_sD_g01*sin(phi_g01) + i_sQ_g01*cos(phi_g01),          

In [9]:
sys_dict['x']

Matrix([
[i_tD_g01],
[i_tQ_g01],
[v_mD_g01],
[v_mQ_g01],
[i_sD_g01],
[i_sQ_g01]])